In [2]:
import pandas as pd

pd.options.display.max_columns = 100
pd.options.display.min_rows = 10
pd.options.display.precision = 3
pd.options.display.float_format = "{:.3f}".format

import numpy as np

np.set_printoptions(precision=3)
np.set_printoptions(suppress=True)
np.set_printoptions(formatter={"float_kind": "{:.3f}".format})

import sys
from pathlib import Path
import cmdstanpy

# ruff: noqa
from psych import _fast
import psych.estimation as est

ROOT_PATH = (
    Path("__file__").resolve().parents[1]
)  # 0 for .py or unsaved notebooks and 1 for .ipynb
sys.path.append(ROOT_PATH.as_posix())

DATA_PATH = ROOT_PATH / "data"
RESULTS_PATH = ROOT_PATH / "results"
MODEL_PATH = ROOT_PATH / "analysis" / "models"
import cmdstanpy

In [41]:
df_resp = pd.read_parquet(DATA_PATH / "COTS_2025_data.parquet")

In [42]:
df_resp["max_score"] = df_resp.groupby("item_id")["score"].transform("max")

# Ensure 1-index for Stan
df_resp["person_id"] = pd.factorize(df_resp["person_id"])[0] + 1
df_resp["item_id"] = pd.factorize(df_resp["item_id"])[0] + 1

In [43]:
df_nr = est.item_calibrations(
    df_resp,
    label_mapper={
        "itemset_id": "setID",
        "person_id": "regID",
        "score": "score",
        "item_id": "itemID",
        "op_theta": "theta",
    },
)

In [106]:
threshold_start = df_resp.groupby("item_id")["max_score"].first().cumsum() + 1
threshold_start = threshold_start.shift(1).replace(np.nan, 1).astype(int).to_numpy()
total_thresholds = df_resp.groupby("item_id")["max_score"].first().sum()

stan_data = {
    "N": len(df_resp),
    "item_id": df_resp["item_id"].to_numpy(),
    "person_id": df_resp["person_id"].to_numpy(),
    "theta": df_resp["op_theta"].to_numpy(),
    "scores": (df_resp["score"] + 1).to_numpy(),
    "categories_per_item": (df_resp["max_score"] + 1).to_numpy(),
    "n_items": df_resp["item_id"].nunique(),
    "threshold_start": threshold_start,
    "total_thresholds": total_thresholds,
    "max_categories": df_resp["max_score"].max()+1,
}


# --- Compile and Run Model ---
model = cmdstanpy.CmdStanModel(
    stan_file= MODEL_PATH / "pcm_fixed_theta.stan",
    cpp_options={'STAN_THREADS': True}
)

# fit = model.sample(
#     data=stan_data,
#     chains=1,  # Start with 1 chain for debugging
#     iter_warmup=10,
#     iter_sampling=100,  # Reduce for testing
#     show_progress=True,
# )

# fit = model.sample(
#     data=stan_data,
#     chains=4,
#     parallel_chains=4,
#     iter_warmup=10,
#     iter_sampling=200,
#     show_progress=True,
# )

fit = model.variational(
    data=stan_data,
    algorithm='meanfield',
    iter=5000,
    draws=5000
)

06:27:47 - cmdstanpy - INFO - Chain [1] start processing
06:28:18 - cmdstanpy - INFO - Chain [1] done processing


In [107]:
nr_thresholds = df_nr.filter(np.arange(1,11)).to_numpy()
nr_thresholds = nr_thresholds.flatten()
nr_thresholds = nr_thresholds[~np.isnan(nr_thresholds)]

np.allclose(nr_thresholds, fit.variational_sample_pd.filter(regex="thresh").median(), atol=.2)
nr_thresholds - fit.variational_sample_pd.filter(regex="thresh").median()

threshold[1]    -0.031
threshold[2]     0.041
threshold[3]     0.001
threshold[4]    -0.015
threshold[5]     0.022
threshold[6]    -0.025
threshold[7]     0.032
threshold[8]     0.057
threshold[9]     0.058
threshold[10]    0.037
threshold[11]   -0.001
threshold[12]   -0.064
threshold[13]    0.010
threshold[14]   -0.009
threshold[15]   -0.002
threshold[16]   -0.006
threshold[17]    0.091
threshold[18]    0.078
threshold[19]    0.096
threshold[20]   -0.039
threshold[21]    0.003
threshold[22]    0.078
threshold[23]    0.075
threshold[24]    0.061
threshold[25]   -0.097
threshold[26]   -0.041
threshold[27]    0.039
threshold[28]   -0.043
threshold[29]   -0.039
threshold[30]    0.053
threshold[31]   -0.076
threshold[32]   -0.080
threshold[33]    0.051
threshold[34]   -0.069
dtype: float64

In [108]:
mle = model.optimize(data=stan_data)

06:29:32 - cmdstanpy - INFO - Chain [1] start processing
06:29:32 - cmdstanpy - INFO - Chain [1] done processing


In [118]:
nr_thresholds - mle.optimized_params_pd.filter(regex="thresh")

,threshold[1],threshold[2],threshold[3],threshold[4],threshold[5],threshold[6],threshold[7],threshold[8],threshold[9],threshold[10],threshold[11],threshold[12],threshold[13],threshold[14],threshold[15],threshold[16],threshold[17],threshold[18],threshold[19],threshold[20],threshold[21],threshold[22],threshold[23],threshold[24],threshold[25],threshold[26],threshold[27],threshold[28],threshold[29],threshold[30],threshold[31],threshold[32],threshold[33],threshold[34]
0,0.000,-0.001,0.001,0.001,0.001,0.000,0.002,-0.000,0.000,0.000,0.000,0.002,0.000,0.001,0.001,0.002,0.002,-0.000,0.011,0.001,-0.003,0.000,0.004,0.003,-0.000,0.001,0.003,0.000,0.002,0.001,0.000,0.004,-0.000,-0.000
